In [43]:
import folium
import requests
import pandas as pd
import numpy as np 
import random
import json
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


### First, we define the url, and start to use REST API and Beautiful Soup

In [44]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [45]:
raw=requests.get(url)
soup = BeautifulSoup(raw.text, 'html.parser' )

In [46]:
table = soup.find('table', class_="wikitable sortable")

### Postal table in raw format

In [47]:
data_temp=pd.DataFrame()
for row in table.find_all('tr')[1:]:
    if row.find_all('td')[1].text!='Not assigned\n':
        data_temp=pd.concat([data_temp,pd.DataFrame([[row.find_all('td')[0].text.replace('\n',''),row.find_all('td')[1].text.replace('\n',''),row.find_all('td')[2].text.replace('\n','')]])])
data_temp.columns=['PostalCode','Borough','Neighborhood']
data_temp.reset_index(drop=True, inplace=True)
# data.sort_values(by='Borough', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)
data_temp


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Postal Table after format:
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [48]:
data_temp.shape

(103, 3)

In [49]:
data=pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])
for row in table.find_all('tr')[1:]:
    if row.find_all('td')[1].text!='Not assigned\n':
        if row.find_all('td')[0].text.replace('\n','')not in list(data['Postal Code']):
            
            data=pd.concat([data,pd.DataFrame([[row.find_all('td')[0].text.replace('\n',''),row.find_all('td')[1].text.replace('\n',''),row.find_all('td')[2].text.replace('\n','')]],columns=['Postal Code','Borough','Neighborhood'])])
#            
        else:
            update=''
            index=data[data['Postal Code']==row.find_all('td')[0].text.replace('\n','')].index[0]

            
            old=data.loc[data['Postal Code']==row.find_all('td')[0].text.replace('\n','')]['Neighborhood'][0]
            new=row.find_all('td')[2].text.replace('\n','')
            update=f'{old},{new}'
#             data.loc[data['Borough']==row.find_all('td')[1].text.replace('\n',''),data['Neighborhood']]=update
            data['Neighborhood'].loc[data['Postal Code']==row.find_all('td')[1].text.replace('\n','')]=update

data=data.reset_index()
data



,index,Postal Code,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,0,M4A,North York,Victoria Village
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,0,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,0,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,0,M4Y,Downtown Toronto,Church and Wellesley
100,0,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,0,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### After processing Borough duplication, only 10 rows left:

In [50]:
data.shape

(103, 4)

## Findings:
### By Grouping Borough column, we can easily see Postal Codes in different zones

In [51]:
data_temp.groupby(['Borough','PostalCode']).sum()

Neighborhood
Borough         PostalCode                                                  
Central Toronto M4N                                            Lawrence Park
                M4P                                         Davisville North
                M4R                       North Toronto West,  Lawrence Park
                M4S                                               Davisville
                M4T                              Moore Park, Summerhill East
...                                                                      ...
York            M6C                                       Humewood-Cedarvale
                M6E                                      Caledonia-Fairbanks
                M6M         Del Ray, Mount Dennis, Keelsdale and Silverthorn
                M6N                            Runnymede, The Junction North
                M9N                                                   Weston

[103 rows x 1 columns]

In [52]:
geo='http://cocl.us/Geospatial_data'

In [53]:
geo_csv=pd.read_csv(geo)
geo_csv

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [54]:
df=pd.merge(data,geo_csv,on=['Postal Code'])
df

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,0,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,0,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,0,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,0,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,0,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [55]:
df.drop(columns=['index'],inplace = True)

In [56]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [58]:
Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto)  
    
Toronto

In [229]:
X=df[['Latitude','Longitude']]
k_means = KMeans(init="k-means++", n_clusters=10, n_init=12)
k_means.fit(X)
k_means_labels = k_means.labels_
df['label']=k_means_labels
k_means_cluster_centers = k_means.cluster_centers_
Centers=pd.DataFrame(k_means_cluster_centers,columns=['Latitude', 'Longitude'])
k_means_labels

array([7, 5, 3, 1, 3, 4, 0, 2, 5, 3, 1, 4, 0, 5, 5, 3, 9, 4, 0, 5, 3, 8,
       0, 9, 3, 8, 7, 2, 1, 5, 3, 8, 7, 2, 1, 5, 3, 8, 7, 2, 1, 3, 3, 8,
       5, 2, 1, 5, 3, 1, 6, 7, 2, 1, 3, 9, 1, 6, 5, 2, 6, 9, 9, 8, 6, 7,
       2, 9, 9, 8, 6, 7, 2, 9, 9, 8, 4, 6, 7, 9, 3, 8, 7, 9, 3, 7, 9, 3,
       4, 6, 7, 3, 3, 4, 6, 0, 3, 3, 4, 3, 5, 4, 4], dtype=int32)

In [220]:
Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
colors_array = cm.rainbow(np.linspace(0, 1, 10))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, borough, neighborhood, labels in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'],df['label']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=rainbow[labels-1],
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(Toronto)

# for lat,lng in zip(Centers['Latitude'],Centers['Longitude']):
#     folium.CircleMarker(
#         [lat, lng],
#         radius=600,
#         popup=label,
#         color='black',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=1,
#         parse_html=False).add_to(Toronto) 
Toronto

In [221]:
address = 'New York city'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 14.09212055, -87.19113829894533.


In [222]:
CLIENT_ID = '5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB' # your Foursquare ID
CLIENT_SECRET = 'ZZQ13GXA2PCBDEANJ0FTW41WNHLKJOEY0OFVEKVI1UDUFVDZ' # your Foursquare Secret
ACCESS_TOKEN = 'PZXYPPFXQLLGT0ICJCP42HBCA52DKQCUGMOKVQ0ZRTULI2TW' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB
CLIENT_SECRET:ZZQ13GXA2PCBDEANJ0FTW41WNHLKJOEY0OFVEKVI1UDUFVDZ


In [226]:
search_query = 'New York'
radius = 500000
print(search_query + ' .... OK!')
# print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
#                                                                neighborhood_latitude, 
#                                                                neighborhood_longitude))

New York .... OK!


In [227]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB&client_secret=ZZQ13GXA2PCBDEANJ0FTW41WNHLKJOEY0OFVEKVI1UDUFVDZ&ll=14.09212055,-87.19113829894533&oauth_token=PZXYPPFXQLLGT0ICJCP42HBCA52DKQCUGMOKVQ0ZRTULI2TW&v=20180604&query=New York&radius=500000&limit=30'

In [228]:
results= requests.get(url).json()
for item in results['response']['venues']:
    print(item['name'])

New York
New Le Petit Macaron
New Port
New Generation Christian School
New Look beauty Salon
New Horizons
Jimmy's Pizza New Location
The New Bar
Sala De Belleza New Look
Newskin
Emporio New York
Mariscos New Fish


In [215]:
json_normalize(results['response']['venues'])


<ipython-input-215-1a40c240cba0>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(results['response']['venues'])


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.labeledLatLngs,location.crossStreet,venuePage.id
0,4a1cc8d1f964a520617b1fe3,Bahnhof Berlin Alexanderplatz,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",v-1607973506,False,Dircksenstr. 2,52.521592,13.411380,158,10179,DE,Alexanderplatz,Berlin,Berlin,Deutschland,"[Dircksenstr. 2, 10179 Berlin, Deutschland]",NaN,NaN,NaN
1,4adcda72f964a520434521e3,Park Inn by Radisson Berlin Alexanderplatz,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1607973506,False,Alexanderplatz 7,52.522825,13.412930,105,10178,DE,NaN,Berlin,Berlin,Deutschland,"[Alexanderplatz 7, 10178 Berlin, Deutschland]","[{'label': 'display', 'lat': 52.522825, 'lng':...",NaN,NaN
2,4a1c8506f964a520457b1fe3,Berlin Hauptbahnhof,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",v-1607973506,False,Europaplatz 1,52.524851,13.369782,2987,10557,DE,NaN,Berlin,Berlin,Deutschland,"[Europaplatz 1 (Washingtonplatz), 10557 Berlin...","[{'label': 'display', 'lat': 52.52485121086412...",Washingtonplatz,NaN
3,4ead71856da1a0e3a47abe19,Hofbräuhaus Berlin,"[{'id': '56aa371ce4b08b9a8d573572', 'name': 'B...",v-1607973506,False,Karl-Liebknecht-Str. 30,52.524576,13.413988,290,10178,DE,NaN,Berlin,Berlin,Deutschland,"[Karl-Liebknecht-Str. 30, 10178 Berlin, Deutsc...","[{'label': 'display', 'lat': 52.524576, 'lng':...",NaN,NaN
4,4af5f0c7f964a52020ff21e3,Bahnhof Berlin Friedrichstraße,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",v-1607973506,False,Georgenstr. 14/17,52.520284,13.387063,1809,10117,DE,NaN,Berlin,Berlin,Deutschland,"[Georgenstr. 14/17, 10117 Berlin, Deutschland]","[{'label': 'display', 'lat': 52.52028438859317...",NaN,NaN
5,4adcda91f964a520bc4b21e3,Berlin Ostbahnhof,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",v-1607973506,False,Koppenstr. 3,52.510600,13.434912,1918,10243,DE,NaN,Berlin,Berlin,Deutschland,"[Koppenstr. 3, 10243 Berlin, Deutschland]",NaN,NaN,NaN
6,4d21b7010901721e2ff29ca5,Berlin,"[{'id': '50aa9e094b90af0d42d5de0d', 'name': 'C...",v-1607973506,False,NaN,52.517422,13.388772,1758,NaN,DE,NaN,NaN,Berlin,Deutschland,[Deutschland],"[{'label': 'display', 'lat': 52.51742216355676...",NaN,NaN
7,4fa8f9984fc614a85bfc54aa,Hotel Indigo Berlin Alexanderplatz,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1607973506,False,Bernhard-Weiß-Str. 5,52.523896,13.417205,322,10178,DE,NaN,Berlin,Berlin,Deutschland,"[Bernhard-Weiß-Str. 5 (Wadzeckstr.), 10178 Ber...","[{'label': 'display', 'lat': 52.5238955, 'lng'...",Wadzeckstr.,NaN
8,50a357c6ebcaea3e9b35bc5c,o2 Shop Berlin 35,"[{'id': '4f04afc02fb6e1c99f3db0bc', 'name': 'M...",v-1607973506,False,Alexanderplatz 3,52.521607,13.414837,91,10178,DE,NaN,Berlin,Berlin,Deutschland,"[Alexanderplatz 3, 10178 Berlin, Deutschland]","[{'label': 'display', 'lat': 52.52160686933147...",NaN,NaN
9,4b37ffeaf964a520144a25e3,Motel One Berlin-Hackescher Markt,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1607973506,False,Dircksenstr. 36,52.523109,13.408626,362,10179,DE,NaN,Berlin,Berlin,Deutschland,"[Dircksenstr. 36 (Rosa-Luxemburg-Str.), 10179 ...","[{'label': 'display', 'lat': 52.52310856190554...",Rosa-Luxemburg-Str.,473955422
